In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2 
import numpy as np
import os

In [ ]:
dossier=os.listdir("/content/drive/MyDrive/Adience_Gender/Adience_Gender_Data_Aug/Femme")
Femme=np.zeros((len(dossier),64,64,3))
Labels_F=[]
for i in range(len(dossier)):
  nom_image='/content/drive/MyDrive/Adience_Gender/Adience_Gender_Data_Aug/Femme/'+dossier[i]
  image = cv2.resize((cv2.imread(nom_image)), (64,64))
  Femme[i]=image
  Labels_F.append(0)

In [ ]:
dossier1=os.listdir("/content/drive/MyDrive/Adience_Gender/Adience_Gender_Data_Aug/Homme")
Homme=np.zeros((len(dossier1),64,64,3))
Labels_H=[]
for i in range(len(dossier1)):
  nom_image='/content/drive/MyDrive/Adience_Gender/Adience_Gender_Data_Aug/Homme/'+dossier1[i]
  image = cv2.resize((cv2.imread(nom_image)), (64,64))
  Homme[i]=image
  Labels_H.append(1)

In [ ]:
def train (x):
  return int(x*0.8)

In [ ]:
images_train_Homme=Homme[:train(len(Homme))]
images_test_Homme=Homme[train(len(Homme)):]
labels_train_Homme=Labels_H[:train(len(Labels_H))]
labels_test_Homme=Labels_H[train(len(Labels_H)):]

images_train_Femme=Femme[:train(len(Femme))]
images_test_Femme=Femme[train(len(Femme)):]
labels_train_Femme=Labels_F[:train(len(Labels_F))]
labels_test_Femme=Labels_F[train(len(Labels_F)):]

In [ ]:
X_train=np.concatenate((images_train_Homme,images_train_Femme),axis=0)
Y_train=np.concatenate((labels_train_Homme,labels_train_Femme),axis=0)
X_test=np.concatenate((images_test_Homme,images_test_Femme),axis=0)
Y_test=np.concatenate((labels_test_Homme,labels_test_Femme),axis=0)

In [ ]:
print(len(X_train))
print(len(Y_train))
print(len(X_test))
print(len(Y_test))

16076
16076
4020
4020


In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-atyy51v9
  Running command git clone --filter=blob:none --quiet https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-atyy51v9
  Resolved https://github.com/rcmalli/keras-vggface.git to commit bee35376e76e35d00aeec503f2f242611a97b38a
  Preparing metadata (setup.py) ... done
  Created wheel for keras-vggface: filename=keras_vggface-0.6-py3-none-any.whl size=8298 sha256=9d24f33788df179a6fd7b04680a85f82c8453b68023e758f6fb4ce4d48c55ea8
  Stored in directory: /tmp/pip-ephem-wheel-cache-tly13gf2/wheels/62/cd/e0/3a2d2fe84d014324aed54d420cab52df65a774d9b296c63996
Successfully built keras-vggface


In [ ]:
!pip install keras_vggface


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras_applications

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.2 MB/s eta 0:00:00


In [ ]:
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace

In [ ]:
vggface = VGGFace(model='vgg16') # or VGGFace() as default


580070376/580070376 [==============================] - 4s 0us/step


In [ ]:
# importing the libraries
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications import VGG19
from keras.layers import Dropout,Dense,Conv2D,GlobalAveragePooling2D
from keras.models import load_model
#from keras.preprocessing import image
num_classes=1
IMAGE_SIZE = [64,64]  # we will keep the image size as (64,64). You can increase the size for better results. 

VGGFace = VGGFace(input_shape = IMAGE_SIZE + [3],weights='vggface',include_top=False)

# this will exclude the initial layers from training phase as there are already been trained.
for layer in VGGFace.layers:
    layer.trainable = False
x = Flatten()(VGGFace.output)

x1 = Dense(num_classes, activation = 'sigmoid')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = VGGFace.input, outputs = x1)

58909280/58909280 [==============================] - 3s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv1_1 (Conv2D)            (None, 64, 64, 64)        1792      
                                                                 
 conv1_2 (Conv2D)            (None, 64, 64, 64)        36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 32, 32, 64)        0         
                                                                 
 conv2_1 (Conv2D)            (None, 32, 32, 128)       73856     
                                                                 
 conv2_2 (Conv2D)            (None, 32, 32, 128)       147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 16, 16, 128)       0     

In [ ]:
import tensorflow as tf

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)


In [ ]:
model.fit(X_train,Y_train, batch_size=100, epochs=1000, validation_data=(X_test,Y_test))

Epoch 1/1000
161/161 [==============================] - 23s 70ms/step - loss: 4.0990 - accuracy: 0.5407 - val_loss: 3.6696 - val_accuracy: 0.5438
Epoch 2/1000
161/161 [==============================] - 9s 56ms/step - loss: 2.6756 - accuracy: 0.6065 - val_loss: 2.8660 - val_accuracy: 0.5826
Epoch 3/1000
161/161 [==============================] - 8s 51ms/step - loss: 2.1940 - accuracy: 0.6320 - val_loss: 2.4523 - val_accuracy: 0.6047
Epoch 4/1000
161/161 [==============================] - 9s 56ms/step - loss: 1.8957 - accuracy: 0.6497 - val_loss: 2.2135 - val_accuracy: 0.6100
Epoch 5/1000
161/161 [==============================] - 8s 50ms/step - loss: 1.6853 - accuracy: 0.6589 - val_loss: 1.9875 - val_accuracy: 0.6192
Epoch 6/1000
161/161 [==============================] - 9s 58ms/step - loss: 1.5168 - accuracy: 0.6679 - val_loss: 1.8265 - val_accuracy: 0.6246
Epoch 7/1000
161/161 [==============================] - 9s 57ms/step - loss: 1.3846 - accuracy: 0.6757 - val_loss: 1.6892 - val_a

In [ ]:
model.evaluate(X_test,Y_test, verbose=2)

126/126 - 2s - loss: 0.8656 - accuracy: 0.6729 - 2s/epoch - 20ms/step
126/126 - 2s - loss: 0.8656 - accuracy: 0.6729 - 2s/epoch - 20ms/step


[0.8655543327331543, 0.6728855967521667]

[0.8655543327331543, 0.6728855967521667]

In [ ]:
model.save('/content/drive/MyDrive/Adience_Gender/VGGFace_Adience_Data_Aug_Gender_Recognition.h5')

# SVM


In [ ]:
pip install keract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import keras
model=keras.models.load_model('/content/drive/MyDrive/Adience_Gender/VGGFace_Adience_Data_Aug_Gender_Recognition.h5')

In [ ]:
from keract import get_activations
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv1_1 (Conv2D)            (None, 64, 64, 64)        1792      
                                                                 
 conv1_2 (Conv2D)            (None, 64, 64, 64)        36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 32, 32, 64)        0         
                                                                 
 conv2_1 (Conv2D)            (None, 32, 32, 128)       73856     
                                                                 
 conv2_2 (Conv2D)            (None, 32, 32, 128)       147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 16, 16, 128)       0     

In [ ]:
activations1 = get_activations(model,X_train[:1000],layer_names='flatten')

In [ ]:
activations2 = get_activations(model,X_train[1000:2000],layer_names='flatten')
activations3 = get_activations(model,X_train[2000:3000],layer_names='flatten')
activations4 = get_activations(model,X_train[3000:4000],layer_names='flatten')
activations5 = get_activations(model,X_train[4000:5000],layer_names='flatten')
activations6 = get_activations(model,X_train[5000:6000],layer_names='flatten')
activations7 = get_activations(model,X_train[6000:7000],layer_names='flatten')
activations8 = get_activations(model,X_train[7000:8000],layer_names='flatten')
activations9 = get_activations(model,X_train[8000:9000],layer_names='flatten')
activations10 = get_activations(model,X_train[9000:10000],layer_names='flatten')
activations11= get_activations(model,X_train[10000:11000],layer_names='flatten')
activations12= get_activations(model,X_train[11000:12000],layer_names='flatten')
activations13= get_activations(model,X_train[12000:13000],layer_names='flatten')
activations14= get_activations(model,X_train[13000:14000],layer_names='flatten')
activations15= get_activations(model,X_train[14000:],layer_names='flatten')


In [ ]:
X_train1= activations1
X_train1= X_train1["flatten"]
X_train2= activations2
X_train2= X_train2["flatten"]
X_train3= activations3
X_train3= X_train3["flatten"]
X_train4= activations4
X_train4= X_train4["flatten"]
X_train5= activations5
X_train5= X_train5["flatten"]
X_train6= activations6
X_train6= X_train6["flatten"]
X_train7= activations7
X_train7= X_train7["flatten"]
X_train8= activations8
X_train8= X_train8["flatten"]
X_train9= activations9
X_train9= X_train9["flatten"]
X_train10= activations10
X_train10= X_train10["flatten"]
X_train11= activations11
X_train11= X_train11["flatten"]
X_train12= activations12
X_train12= X_train12["flatten"]
X_train13= activations13
X_train13= X_train13["flatten"]
X_train14= activations14
X_train14= X_train14["flatten"]
X_train15= activations15
X_train15= X_train15["flatten"]



In [ ]:
X_train =np.concatenate((X_train1,X_train2,X_train3,X_train4,X_train5,X_train6,X_train7,X_train8,X_train9,X_train10,X_train11,X_train12,X_train13,X_train14,X_train15), axis=0)


In [ ]:
activations17 = get_activations(model,X_test[:1000],layer_names='flatten')
activations18 = get_activations(model,X_test[1000:2000],layer_names='flatten')
activations19 = get_activations(model,X_test[2000:3000],layer_names='flatten')
activations20 = get_activations(model,X_test[3000:],layer_names='flatten')

In [ ]:
X_test1= activations17
X_test1= X_test1['flatten']
X_test2= activations18
X_test2= X_test2['flatten']
X_test3= activations19
X_test3= X_test3['flatten']
X_test4= activations20
X_test4= X_test4['flatten']


In [ ]:
X_test =np.concatenate((X_test1,X_test2,X_test3,X_test4), axis=0)


In [ ]:
import pandas as pd
from sklearn.svm import SVC

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
model1 = SVC(kernel='rbf',probability=True, C=100)

In [ ]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
X_train= normalizer.transform(X_train)

In [ ]:
model1.fit(X_train, Y_train)

SVC(C=100, probability=True)

In [ ]:
predictions1 = model1.predict(X_train)

In [ ]:
predictions2 = model1.predict(X_test)

In [ ]:
percentage = model1.score(X_test,Y_test)

In [ ]:
from sklearn.metrics import accuracy_score
print("Dataset: train=%d, test=%d" % (X_train.shape[0], Y_test.shape[0]))
score_train = accuracy_score(Y_train, predictions1)
score_test = accuracy_score(Y_test, predictions2)

print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))



Dataset: train=16076, test=4020
Accuracy: train=99.994, test=50.149


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
res = confusion_matrix(Y_test, predictions2)
for i in range(0,len(res)):
  print(res[i])
print("Confusion Matrix")
print(res)
print(f"Test Set: {len(X_test)}")
print(f"Accuracy = {score_test*100} %")
print(classification_report(Y_test, predictions2))

[   0 2004]
[   0 2016]
Confusion Matrix
[[   0 2004]
 [   0 2016]]
Test Set: 4020
Accuracy = 50.14925373134328 %
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2004
           1       0.50      1.00      0.67      2016

    accuracy                           0.50      4020
   macro avg       0.25      0.50      0.33      4020
weighted avg       0.25      0.50      0.33      4020



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
